<a href="https://colab.research.google.com/github/twilight2001/datasciencecoursera/blob/master/blazingsql_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get Started with Blazing SQL

In this notebook, we will walk through the process for getting BlazingSQL and cuDF running. We will then demonstrate a basic ETL process and query a csv. 

Please click the 'Open in Playground Mode Option ' to run this demo on your own. 

Feel free to add your own code or modify ours as needed.


Background Info:
[BlazingSQL](http://blazingdb.com/#/)
[RAPIDS AI/cuDF](https://rapids.ai/about.html)


# Setup

## Environment Sanity Check 

Once you are in Playground mode, click the Runtime dropdown at the top of the page, then Change Runtime Type and confirm the instance type has GPU selected for 'Hardware Accelerator'.

Check the output of '!nvidia-smi' to make sure you've been allocated a Tesla T4.


In [0]:
!nvidia-smi

Thu Aug 15 23:25:50 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    15W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Sometimes Google Colab will assign a Tesla K80 (which we do not support), so the script below helps confirm you have a T4 GPU attached to your runtime.

In [0]:
import pynvml


pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)

if device_name != b'Tesla T4':
  raise Exception("""
    Unfortunately this instance does not have a T4 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


## Install BlazingSQL + cuDF
This will take roughly a minute and a half.

cuDF is the RAPIDS AI GPU DataFrame library with a Pandas-like API. Learn more about cuDF [here](https://github.com/rapidsai/cudf).

In [0]:
%%writefile bsql-colab.sh

#!/bin/bash


set -eu

wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/env-check.py
echo "Checking for GPU type:"
python env-check.py

if [ ! -f Miniconda3-4.5.4-Linux-x86_64.sh ]; then
    echo "Removing conflicting packages, will replace with RAPIDS compatible versions"
    # remove existing xgboost and dask installs
    pip uninstall -y xgboost dask distributed

    # intall miniconda
    echo "Installing conda"
    wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
    chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
    bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
    
    echo "Installing RAPIDS packages"
    echo "Please standby, this will take a few minutes..."
    # install RAPIDS packages
    conda install -y --prefix /usr/local \
      -c rapidsai/label/xgboost -c rapidsai -c nvidia -c conda-forge \
      python=3.6 cudatoolkit=10.0 \
      cudf=0.9.* cuml=0.9.* cugraph=0.9.* gcsfs pynvml \
      dask-cudf=0.9.* \
      rapidsai/label/xgboost::xgboost=>0.9
      
    echo "Copying shared object files to /usr/lib"
    # copy .so files to /usr/lib, where Colab's Python looks for libs
    cp /usr/local/lib/libcudf.so /usr/lib/libcudf.so
    cp /usr/local/lib/librmm.so /usr/lib/librmm.so
    cp /usr/local/lib/libxgboost.so /usr/lib/libxgboost.so
    cp /usr/local/lib/libnccl.so /usr/lib/libnccl.so
    conda install -y --prefix /usr/local -c rapidsai -c nvidia -c conda-forge -c felipeblazing/label/cuda10.0 python=3.6 cudatoolkit=10.0 blazingsql-ral blazingsql-orchestrator blazingsql-calcite blazingsql-python
    pip install flatbuffers
fi
echo ""
echo "*********************************************"
echo "Your Google Colab instance is RAPIDS ready!"
echo "*********************************************"



echo ""
echo "*********************************************"
echo "Your Google Colab instance is BlazingSQL ready!"
echo "*********************************************"


Overwriting bsql-colab.sh


In [0]:
%%time
!bash bsql-colab.sh

File ‘env-check.py’ already there; not retrieving.

Checking for GPU type:
*********************************************
Woo! Your instance has the right kind of GPU!
*********************************************


*********************************************
Your Google Colab instance is RAPIDS ready!
*********************************************

*********************************************
Your Google Colab instance is BlazingSQL ready!
*********************************************
CPU times: user 7.55 ms, sys: 12.2 ms, total: 19.7 ms
Wall time: 580 ms


# You are ready to go with BlazingSQL!
Nice Job! Now lets see how it works.

# Import packages and create Blazing Context
You can think of the BlazingContext much like a Spark Context. This is where information such as FileSystems you have registered, Tables you have created will be stored.

If you have issues running this cell, restart runtime and try running it again.


In [0]:
import sys, os, time

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

#Standup the BlazingSQL Services - We are working on removing the need to call these functions and just initializing them in BlazingContext
import subprocess
subprocess.Popen(['blazingsql-orchestrator', '9100', '8889', '127.0.0.1', '8890'],stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
subprocess.Popen(['java', '-jar', '/usr/local/lib/blazingsql-algebra.jar', '-p', '8890'])
import pyblazing.apiv2.context as cont
cont.runRal()
time.sleep(1) #Wait for service to start.

In [0]:
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

Already connected to the Orchestrator


# Read CSV
First we uploaded a CSV file through the Google Colab interface. 

Then we use cuDF to read the CSV file. This gives us a GPU DataFrame (GDF).

To learn more about the GDF and how it enables end to end workloads on rapids, read [this](https://blog.blazingdb.com/blazingsql-part-1-the-gpu-dataframe-gdf-and-cudf-in-rapids-ai-96ec15102240).

In [0]:
#Download the test CSV
!wget 'https://s3.amazonaws.com/blazingsql-colab/Music.csv'

--2019-08-15 23:32:28--  https://s3.amazonaws.com/blazingsql-colab/Music.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.107.206
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.107.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10473 (10K) [text/csv]
Saving to: ‘Music.csv’

Music.csv           100%[===================>]  10.23K  --.-KB/s    in 0s      

2019-08-15 23:32:29 (155 MB/s) - ‘Music.csv’ saved [10473/10473]



In [0]:
gdf = cudf.read_csv('Music.csv')

# Create a Table

Now we just need to create a table. Apache Parquet is a great, open source and distributed file format built for systems like HDFS. Apache Parquet files also have metadata which self describes the schema, making import a cinch!

In [0]:
bc.create_table('music', gdf)

# Query a Table
That's it! Now you can write a SQL query and the data will get processed on the GPU with BlazingSQL, and the output will be a GPU DataFrame (GDF) inside RAPIDS!

In [0]:
#Query
result = bc.sql('SELECT * FROM main.music').get()

#Get GDF
result_gdf = result.columns

#Print GDF
print(result_gdf)

                                  ARTIST RATING  ...          LOCATION FESTIVAL_SET
0                            Arcade Fire   10.0  ...         Las Vegas          1.0
1                                Justice   10.0  ...         Las Vegas          1.0
2               Florence and The Machine   10.0  ...         Las Vegas          1.0
3                                 Odesza   10.0  ...             Indio          1.0
4                               Bon Iver   10.0  ...             Indio          1.0
5            LA Philharmonic + Sigur Ros   10.0  ...                LA          0.0
6                              Sigur Ros   10.0  ...             Malmo          0.0
7                            Arcade Fire   10.0  ...             Indio          1.0
8                                 Escort    9.0  ...     San Francisco          0.0
9                                Phoenix    9.0  ...          Berkeley          0.0
10                              Jamie XX    9.0  ...  Golden Gate Park      

And... thats it! You are now live with BlazingSQL. Check out our [docs](https://docs.blazingdb.com) to get fancy as well as to learn more about how BlazingSQL works with the rest of [RAPIDS AI](https://rapids.ai/). 